In [2]:
import sqlite3
from tkinter import *
from tkinter import messagebox

########## BANCO DE DADOS ##########

conexao = sqlite3.connect("Estoque.db")
cursor = conexao.cursor()

# Criação da tabela (se não existir)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Estoque (
    Id INTEGER PRIMARY KEY AUTOINCREMENT,
    Produto TEXT NOT NULL,
    Quantidade INTEGER NOT NULL,
    DataCompra TEXT NOT NULL,
    Lote INTEGER NOT NULL,
    Marca TEXT NOT NULL
)
''')
conexao.commit()

########## FUNÇÕES ##########

def adicionar_produto():
    try:
        nome = nome_produto.get()
        quantidade = int(qtde_produto.get())
        data = data_produto.get()
        lote = int(lote_produto.get())
        marca = marca_produto.get()

        # Verifica se produto + lote + marca já existem
        cursor.execute('''SELECT Id, Quantidade FROM Estoque WHERE Produto = ? AND Lote = ? AND Marca = ?''', (nome, lote, marca))
        resultado = cursor.fetchone()

        if resultado:
            id_existente, quantidade_atual = resultado
            nova_qtde = quantidade_atual + quantidade
            cursor.execute('''UPDATE Estoque SET Quantidade = ? WHERE Id = ?''', (nova_qtde, id_existente))
            caixa_texto.delete("1.0", END)
            caixa_texto.insert("1.0", f"{quantidade} unidades adicionadas ao produto existente.")
        else:
            cursor.execute('''INSERT INTO Estoque (Produto, Quantidade, DataCompra, Lote, Marca) VALUES (?, ?, ?, ?, ?)''',
                           (nome, quantidade, data, lote, marca))
            caixa_texto.delete("1.0", END)
            caixa_texto.insert("1.0", f"{nome} inserido como novo produto.")

        conexao.commit()

    except Exception as e:
        messagebox.showerror("Erro", str(e))

def deletar_produto():
    if len(nome_produto.get()) < 2:
        caixa_texto.delete("1.0", END)
        caixa_texto.insert("1.0", f"Nome do Produto inválido!")
        return

    cursor.execute('''DELETE FROM Estoque WHERE Produto=?''', (nome_produto.get(),))
    conexao.commit()
    caixa_texto.delete("1.0", END)
    caixa_texto.insert("1.0", f"{nome_produto.get()} deletado com sucesso!")

def consumir_produto():
    try:
        cursor.execute('''
        UPDATE Estoque SET Quantidade = Quantidade - ?
        WHERE Produto = ? AND Lote = ? AND Marca = ?''',
        (int(qtde_produto.get()), nome_produto.get(), int(lote_produto.get()), marca_produto.get()))

        conexao.commit()
        caixa_texto.delete("1.0", END)
        caixa_texto.insert("1.0", f"{qtde_produto.get()} unidades de {nome_produto.get()} consumidas!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def visualizar_produto():
    cursor.execute('''SELECT * FROM Estoque WHERE Produto = ?''', (nome_produto.get(),))
    valores = cursor.fetchall()

    texto = ""
    for id_produto, nome, quantidade, data_compra, lote, marca in valores:
        texto += f"""
-----
Produto: {nome}
Quantidade: {quantidade}
Data da compra: {data_compra}
Lote: {lote}
Marca: {marca}
"""
    caixa_texto.delete("1.0", END)
    caixa_texto.insert("1.0", texto if texto else "Produto não encontrado")

########## INTERFACE ##########

window = Tk()
window.title("Controle de Estoque")
window.geometry("711x646")
window.configure(bg="#002f6c")

Label(window, text="Controle de estoque artigos eletrônicos Garcia", fg="white", bg="#002f6c", font=("Helvetica", 16, "bold")).pack(pady=10)

FrameCampos = Frame(window, bg="#002f6c")
FrameCampos.pack(pady=10)

# Entradas
campos = ["Nome do produto:", "Data da compra:", "Marca:", "Lote:", "Quantidade:"]
entries = []

for i, campo in enumerate(campos):
    Label(FrameCampos, text=campo, bg="#002f6c", fg="white", anchor="w", width=18).grid(row=i, column=0, sticky=W, padx=10, pady=5)
    ent = Entry(FrameCampos, width=40)
    ent.grid(row=i, column=1, padx=10, pady=5)
    entries.append(ent)

nome_produto, data_produto, marca_produto, lote_produto, qtde_produto = entries

# Botões em uma única linha
FrameBotoes = Frame(window, bg="#002f6c")
FrameBotoes.pack(pady=20)

Button(FrameBotoes, text="Adicionar Produto", command=adicionar_produto, bg="orange").grid(row=0, column=0, padx=10, pady=5)
Button(FrameBotoes, text="Registrar venda Produto", command=consumir_produto, bg="orange").grid(row=0, column=1, padx=10, pady=5)
Button(FrameBotoes, text="Deletar Produto", command=deletar_produto, bg="orange").grid(row=0, column=2, padx=10, pady=5)
Button(FrameBotoes, text="Procurar Produto", command=visualizar_produto, bg="orange").grid(row=0, column=3, padx=10, pady=5)

# Caixa de texto resultado
caixa_texto = Text(window, height=7, width=80)
caixa_texto.pack(pady=10)

window.mainloop()
cursor.close()
conexao.close()



